In [1]:
import sys

sys.path.insert(0, '..')

In [2]:
from morpheus.settings import settings
import os
settings.MORPHEUS_MODFLOW_LOCAL_DATA = (os.path.join(os.getcwd(), 'data'))
settings.__dict__

{'ENV': 'production',
 'SECRET_KEY': 'bbf8485d7ccec0d7fd9027da3d065e509fb55db23c70774c',
 'KEYCLOAK_CLIENT_ID': 'admin',
 'KEYCLOAK_CLIENT_SECRET': 'BaR9zZ7yhcDlBbfu9cH43xu4rQg82S17',
 'KEYCLOAK_REALM': 'inowas',
 'KEYCLOAK_SERVER_URL': 'http://identity.inowas.localhost',
 'MONGO_HOST': 'localhost',
 'MONGO_PORT': 27017,
 'MONGO_USER': 'dev',
 'MONGO_PASSWORD': 'dev',
 'MONGO_MODFLOW_DATABASE': 'modflow_data',
 'MONGO_SENSOR_DATABASE': 'sensor_data',
 'MORPHEUS_MODFLOW_LOCAL_DATA': '/Users/ralf/Projects/inowas/morpheus/src/backend/src/notebooks/data',
 'MORPHEUS_SENSORS_UIT_FTP_HOST': 'ftp',
 'MORPHEUS_SENSORS_UIT_FTP_USER': 'dev',
 'MORPHEUS_SENSORS_UIT_FTP_PASSWORD': 'dev',
 'MORPHEUS_SENSORS_UIT_FTP_PATH': '/',
 'MORPHEUS_SENSORS_LOCAL_DATA': '/Users/ralf/Projects/inowas/morpheus/infrastructure/local/data/morpheus/sensors',
 'OPENAPI_LOCAL_SPEC_FILE': '/Users/ralf/Projects/inowas/morpheus/src/backend/src/morpheus/openapi.yml',
 'OPENAPI_BUNDLED_SPEC_FILE': '/Users/ralf/Projects/inow

## Setup the model area

In [3]:
from morpheus.modflow.types.discretization.spatial.GridCells import GridCells
from morpheus.modflow.types.discretization import SpatialDiscretization
from morpheus.modflow.types.discretization.spatial import Rotation, LengthUnit, Grid, Crs
from morpheus.modflow.types.geometry import Polygon, LineString

polygon = Polygon(
  type='Polygon',
  coordinates=[[
    (13.922514437551428, 50.964720483303836),
    (13.925250781947113, 50.965228748412386),
    (13.925036413951403, 50.96623732041704),
    (13.92222441026388, 50.96629040370362),
    (13.922514437551428, 50.964720483303836)
  ]]
)

rotation = Rotation(0)
length_unit = LengthUnit.meters()
x_coordinates = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
y_coordinates = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

grid = Grid.from_polygon_with_relative_coordinates(
  polygon=polygon,
  x_coordinates=x_coordinates,
  y_coordinates=y_coordinates,
  rotation=rotation,
)

spatial_discretization = SpatialDiscretization(
  geometry=polygon,
  grid=grid,
  affected_cells=GridCells.from_polygon(polygon=polygon, grid=grid),
  crs=Crs.from_str('EPSG:4326')
)

spatial_discretization



SpatialDiscretization(geometry=Polygon(coordinates=[[(13.922514437551428, 50.964720483303836), (13.925250781947113, 50.965228748412386), (13.925036413951403, 50.96623732041704), (13.92222441026388, 50.96629040370362), (13.922514437551428, 50.964720483303836)]], type='Polygon'), grid=Grid(origin=Point(coordinates=(13.922224410263878, 50.9662904037036), type='Point'), x_distances=[33.68941547288559, 33.68941547288559, 33.6894154728856, 33.689415472885585, 33.6894154728856, 33.6894154728856, 33.68941547288557, 33.6894154728856, 33.68941547288563, 33.68941547288557], y_distances=[27.74944856585935, 27.74944856585935, 27.749448565859346, 27.749448565859353, 27.74944856585934, 27.749448565859353, 27.749448565859325, 27.74944856585938, 27.749448565859325, 27.749448565859353], rotation=Rotation(value=0), length_unit=LengthUnit(unit=2)), affected_cells=GridCells(shape=(10, 10), data=[GridCell(x=0, y=0, value=True), GridCell(x=0, y=1, value=True), GridCell(x=0, y=2, value=True), GridCell(x=0, y=

## Setup Time Discretization

In [4]:
from morpheus.modflow.types.discretization.time.TimeUnit import TimeUnit
from morpheus.modflow.types.discretization.time.Stressperiods import StartDateTime, StressPeriodCollection, \
  StressPeriod, NumberOfTimeSteps, TimeStepMultiplier, IsSteadyState
from morpheus.modflow.types.discretization.time import TimeDiscretization
from datetime import datetime

time_discretization = TimeDiscretization(
  start_date_time=StartDateTime.from_datetime(datetime(2020, 1, 1)),
  end_date_time=StartDateTime.from_datetime(datetime(2020, 12, 31)),
  stress_periods=StressPeriodCollection([
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 1, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.yes()
    ),
  ]),
  time_unit=TimeUnit.days()
)

time_discretization

TimeDiscretization(start_date_time=StartDateTime(value=datetime.datetime(2020, 1, 1, 0, 0)), end_date_time=StartDateTime(value=datetime.datetime(2020, 12, 31, 0, 0)), stress_periods=StressPeriodCollection(values=[StressPeriod(start_date_time=StartDateTime(value=datetime.datetime(2020, 1, 1, 0, 0)), number_of_time_steps=NumberOfTimeSteps(value=1), time_step_multiplier=TimeStepMultiplier(value=1), steady_state=IsSteadyState(value=True))]), time_unit=TimeUnit(unit=4))

# Setup Modflow Model

In [5]:
from morpheus.modflow.types.ModflowModel import ModflowModel

modflow_model = ModflowModel.new()
modflow_model = modflow_model.with_updated_time_discretization(time_discretization)
modflow_model = modflow_model.with_updated_spatial_discretization(spatial_discretization=spatial_discretization)

In [6]:
import folium

m = folium.Map(tiles="cartodbpositron", crs="EPSG3857", zoom_start=12, location=[50.965, 13.922])
m.add_child(folium.GeoJson(polygon.as_geojson()))

m.fit_bounds(m.get_bounds())
m

# Setup constant head boundary with 1 observation

In [7]:
from morpheus.modflow.types.boundaries.ConstantHeadObservation import ObservationId, HeadValue, ConstantHeadRawDataItem
from morpheus.modflow.types.geometry import Point
from morpheus.modflow.types.boundaries.Boundary import BoundaryName
from morpheus.modflow.types.soil_model import LayerId
from morpheus.modflow.types.boundaries.Boundary import ConstantHeadBoundary, ConstantHeadObservation

layers_list = [LayerId.new()]
constant_head = ConstantHeadBoundary.from_geometry(
  name=BoundaryName('constant_head'),
  geometry=LineString([
    (13.9223, 50.9662),
    (13.9250, 50.96615),
  ]),
  grid=grid,
  affected_layers=[modflow_model.soil_model.layers[0].id],
  observations=[
    ConstantHeadObservation(
      observation_id=ObservationId.new(),
      geometry=Point((13.9223, 50.9662)),
      raw_data=[
        ConstantHeadRawDataItem(
          date_time=StartDateTime.from_datetime(datetime(2019, 1, 1)),
          head=HeadValue.from_float(100),
        ),
        ConstantHeadRawDataItem(
          date_time=StartDateTime.from_datetime(datetime(2021, 12, 31)),
          head=HeadValue.from_float(120),
        ),
      ]
    )
  ]
)

m.add_child(folium.GeoJson(constant_head.as_geojson()))
m.add_child(folium.GeoJson(constant_head.affected_cells.as_geojson(grid).as_geojson()))
for constant_head.observation in constant_head.observations:
  m.add_child(folium.GeoJson(constant_head.observation.as_geojson()))
m.fit_bounds(m.get_bounds())
m

In [8]:
from morpheus.modflow.types.boundaries.Boundary import BoundaryCollection

boundary_collection = BoundaryCollection.new()
boundary_collection.add_boundary(constant_head)

modflow_model = modflow_model.with_updated_boundaries(boundaries=boundary_collection)

## Calculate

In [9]:
from morpheus.modflow.types.calculation.CalculationProfile import CalculationProfile, CalculationEngineType
from morpheus.modflow.types.calculation.Calculation import Calculation
from morpheus.modflow.infrastructure.calculation.modflow_2005.Mf2005CalculationEngineSettings import \
  Mf2005CalculationEngineSettings

calculation_engine_settings = Mf2005CalculationEngineSettings.default()
calculation_profile = CalculationProfile.new(engine_type=CalculationEngineType.MF2005,
                                             engine_settings=calculation_engine_settings.to_dict())
calculation = Calculation.new(modflow_model=modflow_model, calculation_profile=calculation_profile)

In [10]:
calculation = Calculation.from_dict(calculation.to_dict())
calculation.calculation_id

CalculationId(value='3df68a99-d0bb-4974-b8d8-642747d4cfc1')

In [11]:
from morpheus.modflow.infrastructure.calculation.service.CalculationService import CalculationService

calculation_service = CalculationService.from_calculation(calculation)
calculation_service.calculate()

FloPy is using the following executable to run the model: ../../../../.venv/bin/mf2005

                                  MODFLOW-2005     
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 1.12.00 2/3/2017                        

 Using NAME file: 894bc595-5aa6-4eec-a450-6f3de41bdb5a.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2023/12/14 11:33:25

 Solving:  Stress period:     1    Time step:     1    Ground-Water Flow Eqn.
 Run end date and time (yyyy/mm/dd hh:mm:ss): 2023/12/14 11:33:25
 Elapsed run time:  0.025 Seconds

  Normal termination of simulation


In [12]:
calculation_service.get_result()

CalculationResult(type=<CalculationResultType.SUCCESS: 'success'>, message='', files=['894bc595-5aa6-4eec-a450-6f3de41bdb5a.chd', '894bc595-5aa6-4eec-a450-6f3de41bdb5a.dis', '894bc595-5aa6-4eec-a450-6f3de41bdb5a.oc', '894bc595-5aa6-4eec-a450-6f3de41bdb5a.bas', '894bc595-5aa6-4eec-a450-6f3de41bdb5a.nam', '894bc595-5aa6-4eec-a450-6f3de41bdb5a.lpf', '894bc595-5aa6-4eec-a450-6f3de41bdb5a.list', '894bc595-5aa6-4eec-a450-6f3de41bdb5a.pcg', '894bc595-5aa6-4eec-a450-6f3de41bdb5a.hds'], head_results=AvailableResults(times=[365.0], kstpkper=[(0, 0)], number_of_layers=1), drawdown_results=None, budget_results=AvailableResults(times=[365.0], kstpkper=[(0, 0)], number_of_layers=0), concentration_results=None)

In [13]:
head = calculation_service.read_head(0, 0)

In [14]:
import simplejson as json

json = json.dumps(head, ignore_nan=True)
json

'[[113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, null], [113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, null], [113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547], [113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547], [113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547, 113.33300018310547], [null, 113.33300

In [15]:
calculation_service.read_drawdown(0, 0)

[]

In [16]:
calculation_service.read_budget(0, False)

{'STORAGE_IN': 0.0,
 'CONSTANT_HEAD_IN': 0.0,
 'TOTAL_IN': 0.0,
 'STORAGE_OUT': -0.0,
 'CONSTANT_HEAD_OUT': -0.0018196,
 'TOTAL_OUT': -0.0018196,
 'IN-OUT': -0.0018196,
 'PERCENT_DISCREPANCY': -200.0,
 'tslen': 365.0}

In [17]:
calculation_service.read_budget(0, True)

{'STORAGE_IN': 0.0,
 'CONSTANT_HEAD_IN': 0.0,
 'TOTAL_IN': 0.0,
 'STORAGE_OUT': -0.0,
 'CONSTANT_HEAD_OUT': -4.9852e-06,
 'TOTAL_OUT': -4.9852e-06,
 'IN-OUT': -4.9852e-06,
 'PERCENT_DISCREPANCY': -200.0,
 'tslen': 365.0}

In [18]:
calculation_service.get_result()

CalculationResult(type=<CalculationResultType.SUCCESS: 'success'>, message='', files=['894bc595-5aa6-4eec-a450-6f3de41bdb5a.chd', '894bc595-5aa6-4eec-a450-6f3de41bdb5a.dis', '894bc595-5aa6-4eec-a450-6f3de41bdb5a.oc', '894bc595-5aa6-4eec-a450-6f3de41bdb5a.bas', '894bc595-5aa6-4eec-a450-6f3de41bdb5a.nam', '894bc595-5aa6-4eec-a450-6f3de41bdb5a.lpf', '894bc595-5aa6-4eec-a450-6f3de41bdb5a.list', '894bc595-5aa6-4eec-a450-6f3de41bdb5a.pcg', '894bc595-5aa6-4eec-a450-6f3de41bdb5a.hds'], head_results=AvailableResults(times=[365.0], kstpkper=[(0, 0)], number_of_layers=1), drawdown_results=None, budget_results=AvailableResults(times=[365.0], kstpkper=[(0, 0)], number_of_layers=0), concentration_results=None)